# Interpretation - Feature Importance + Partial Dependence + Project Improved Model Delivery

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/15_interpretation_error_analysis_project.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Generate model interpretation artifacts (permutation importance, PDP/ICE)
2. Conduct error analysis to find systematic failure segments
3. Communicate model behavior honestly (limits, caveats, instability)
4. Deliver a project improved model with interpretation and error analysis
5. Use Gemini to draft explanation text, then tighten it to evidence

---

## 1. Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)

print("✓ Setup complete!")

## 2. Load Data and Train Champion Model

In [ ]:
# Load dataset
california = fetch_california_housing(as_frame=True)
df = california.frame

X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

# Split data
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

In [ ]:
# Train champion model (Random Forest for interpretation)
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_SEED, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predictions
y_val_pred = rf_model.predict(X_val)

# Evaluation
mae = mean_absolute_error(y_val, y_val_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
r2 = r2_score(y_val, y_val_pred)

print("\n=== CHAMPION MODEL PERFORMANCE ===")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

## 3. Permutation Feature Importance

### 3.1 Compute Permutation Importance

In [ ]:
# Compute permutation importance on validation set
perm_importance = permutation_importance(
    rf_model, X_val, y_val, 
    n_repeats=10, 
    random_state=RANDOM_SEED,
    scoring='neg_mean_absolute_error'
)

# Create importance DataFrame
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("\n=== PERMUTATION FEATURE IMPORTANCE ===")
print(importance_df)

### 3.2 Visualize Importance

In [ ]:
# Plot permutation importance
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(importance_df['feature'], importance_df['importance_mean'], xerr=importance_df['importance_std'])
ax.set_xlabel('Permutation Importance (decrease in MAE)')
ax.set_title('Feature Importance - Random Forest Model')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Create permutation importance and write 3 evidence-based bullets about feature importance.

**Instructions:**
1. Review the permutation importance results above
2. Identify the top 3 most important features
3. Write 3 evidence-based interpretation bullets

---

### YOUR INTERPRETATION HERE:

**Finding 1:**  
[Evidence-based interpretation of most important feature]

**Finding 2:**  
[Evidence-based interpretation of second feature]

**Finding 3:**  
[Evidence-based interpretation or pattern]

---

## 4. Partial Dependence Plots (PDP)

### 4.1 Create PDP for Top Features

In [ ]:
# Select top 4 features for PDP
top_features = importance_df.head(4)['feature'].tolist()

print(f"Creating PDP for: {top_features}")

# Create partial dependence plots
fig, ax = plt.subplots(figsize=(14, 10))
display = PartialDependenceDisplay.from_estimator(
    rf_model, X_val, top_features, 
    ax=ax, 
    kind="average",
    random_state=RANDOM_SEED
)
plt.suptitle('Partial Dependence Plots - Top 4 Features', fontsize=16)
plt.tight_layout()
plt.show()

### 4.2 Individual Conditional Expectation (ICE) Plots

In [ ]:
# Create ICE plots for top feature
top_feature = top_features[0]

fig, ax = plt.subplots(figsize=(10, 6))
display = PartialDependenceDisplay.from_estimator(
    rf_model, X_val, [top_feature],
    kind="both",  # Shows both average PDP and individual ICE curves
    ax=ax,
    random_state=RANDOM_SEED,
    ice_lines_kw={"alpha": 0.1}
)
plt.suptitle(f'ICE Plot for {top_feature}', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\n⚠️ Interpretation Notes:")
print(f"  - PDP shows average effect of {top_feature} on predictions")
print(f"  - ICE curves show effect for individual samples")
print(f"  - Wide spread in ICE = interactions with other features")

## 5. Error Analysis

### 5.1 Residual Analysis

In [ ]:
# Compute residuals
residuals = y_val - y_val_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residual plot
axes[0].scatter(y_val_pred, residuals, alpha=0.3)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('Predicted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residual Plot')

# Residual histogram
axes[1].hist(residuals, bins=50, edgecolor='black')
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution')

plt.tight_layout()
plt.show()

print(f"\nResidual Statistics:")
print(f"  Mean: {residuals.mean():.4f}")
print(f"  Std: {residuals.std():.4f}")
print(f"  Median: {residuals.median():.4f}")

### 5.2 Segment Error Analysis

In [ ]:
# Analyze errors by segments
# Create price segments
X_val_analysis = X_val.copy()
X_val_analysis['y_true'] = y_val.values
X_val_analysis['y_pred'] = y_val_pred
X_val_analysis['abs_error'] = np.abs(residuals)

# Segment by top feature
top_feature = importance_df.iloc[0]['feature']
X_val_analysis[f'{top_feature}_segment'] = pd.qcut(X_val_analysis[top_feature], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Error by segment
segment_errors = X_val_analysis.groupby(f'{top_feature}_segment').agg({
    'abs_error': ['mean', 'median', 'std'],
    'y_true': 'count'
}).round(4)

print(f"\n=== ERROR ANALYSIS BY {top_feature} SEGMENT ===")
print(segment_errors)

In [ ]:
# Visualize errors by segment
fig, ax = plt.subplots(figsize=(10, 6))
X_val_analysis.boxplot(column='abs_error', by=f'{top_feature}_segment', ax=ax)
ax.set_xlabel(f'{top_feature} Quartile')
ax.set_ylabel('Absolute Error')
ax.set_title(f'Error Distribution by {top_feature} Segment')
plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Run segment error analysis and identify one failure segment.

**Instructions:**
1. Review the segment error analysis above
2. Identify which segment has the highest error
3. Propose a hypothesis for why this segment performs poorly
4. Suggest one potential improvement

---

### YOUR SEGMENT ANALYSIS HERE:

**Highest Error Segment:**  
[Which segment and why?]

**Hypothesis:**  
[Why does this segment have higher errors?]

**Potential Improvement:**  
[What could help reduce errors in this segment?]

---

## 6. Interpretation Narrative Template (Evidence-Based)

### 6.1 Model Strengths

**Model Strengths (Evidence-Based):**

1. **Overall Performance**: The Random Forest model achieves an R² of [X.XX] on the validation set, explaining [XX]% of variance in housing prices.

2. **Key Drivers**: Feature importance analysis reveals that [Top Feature] is the strongest predictor, with a permutation importance of [X.XX], followed by [Second Feature].

3. **Predictive Patterns**: Partial dependence plots show [describe pattern] relationship between [feature] and predictions.

---

### 6.2 Model Limitations

**Model Limitations (Honest Assessment):**

1. **Segment Performance Gaps**: Error analysis reveals systematically higher errors in [segment description], with mean absolute error of [X.XX] compared to overall MAE of [X.XX].

2. **Feature Interactions**: ICE plots show substantial variation in individual effects, suggesting complex interactions that the model may not fully capture.

3. **Residual Patterns**: Residual analysis indicates [describe pattern], suggesting [interpretation].

4. **Generalization Concerns**: The model is trained on [timeframe/geography], and may not generalize to [different context].

---

### 6.3 Recommendations

**Recommendations:**

1. **Use with Caution**: Apply elevated scrutiny to predictions for [high-error segment].

2. **Feature Engineering**: Consider engineering additional features to capture [identified interaction].

3. **Model Monitoring**: Track permutation importance stability over time to detect feature drift.

4. **Error Bounds**: When communicating predictions, include confidence intervals, especially for [segment].

---

## 7. Project Milestone 3 Scaffold

### 7.1 Deliverable Checklist

**Project Milestone 3 Requirements:**

- [ ] Updated model comparison table (baseline vs improved models)
- [ ] Champion model selection with justification
- [ ] Permutation importance plot and interpretation
- [ ] PDP/ICE plots for top 3-4 features
- [ ] Segment error analysis with findings
- [ ] Evidence-based interpretation narrative
- [ ] Model limitations section (honest assessment)
- [ ] Next steps and recommendations

---

## 8. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Permutation Importance**: Model-agnostic method to measure feature importance
2. **Partial Dependence**: Visualizing average effect of features on predictions
3. **ICE Plots**: Understanding individual-level feature effects and interactions
4. **Error Analysis**: Finding systematic failure patterns through segmentation
5. **Honest Communication**: Evidence-based interpretation with clear limitations

### Interpretation Best Practices:

- ✓ Always compute importance on validation/test data, not training data
- ✓ Report standard deviations to show stability
- ✓ Check for correlated features when interpreting importance
- ✓ Use PDP cautiously when features are highly correlated
- ✓ Conduct segment analysis to find failure modes
- ✓ Be honest about limitations and uncertainties

### Remember:

> **"Interpretation is about honest communication, not selling the model."**  
> Report what you find, including limitations and failure modes.

---

## Bibliography

- scikit-learn User Guide: [Inspection Tools](https://scikit-learn.org/stable/inspection.html) (permutation importance, partial dependence)
- Molnar, C. (2022). *Interpretable Machine Learning*. [Online book](https://christophm.github.io/interpretable-ml-book/)
- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* (ISLP). Springer.
- Breiman, L. (2001). "Random Forests." *Machine Learning*, 45(1), 5-32.

---




<center>

Thank you!

</center>